In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

## load data

In [124]:
csv_file = '../catalan_data_course/catalan-juvenile-recidivism-subset.csv'
df = pd.read_csv(csv_file, header=0)

In [125]:
df.head()

,id,V1_sex,V4_area_origin,V6_province,V8_age,V9_age_at_program_end,V11_criminal_record,V12_n_criminal_record,V13_n_crime_cat,V15_main_crime_cat,...,V28_days_from_crime_to_program,V29_program_duration,V115_RECID2015_recid,V10_date_of_birth_year,V10_date_of_birth_month,V22_main_crime_date_year,V22_main_crime_date_month,V30_program_start_year,V30_program_start_month,V31_program_end_month
0,691,male,Spain,Lleida,15,17,1,1-2,3+,Against People,...,587.0,132,1,1992,11,2008,8,2010,4,8
1,956,female,Spain,Barcelona,14,15,1,1-2,2,Against People,...,107.0,258,1,1995,7,2009,8,2009,12,8
2,2560,male,Maghreb,Barcelona,15,16,1,3-5,3+,Against People,...,135.0,85,1,1994,1,2009,11,2010,3,6
3,2018,male,Maghreb,Girona,14,15,1,5+,2,Against People,...,176.0,271,1,1994,4,2009,1,2009,6,3
4,1650,male,Spain,Tarragona,17,20,1,5+,3+,Against People,...,261.0,533,1,1990,9,2008,9,2009,5,11


In [126]:
df.shape

(4652, 29)

In [127]:
df.isnull().sum()

id                                0
V1_sex                            0
V4_area_origin                    0
V6_province                       0
V8_age                            0
V9_age_at_program_end             0
V11_criminal_record               0
V12_n_criminal_record             0
V13_n_crime_cat                   0
V15_main_crime_cat                0
V16_violent_crime                 0
V17_crime_classification          0
V19_committed_crime               0
V20_n_juvenile_records            0
V21_n_crime                       0
V23_territory_of_execution        0
V24_finished_program              0
V26_finished_measure_grouped      0
V27_program_duration_cat          0
V28_days_from_crime_to_program    0
V29_program_duration              0
V115_RECID2015_recid              0
V10_date_of_birth_year            0
V10_date_of_birth_month           0
V22_main_crime_date_year          0
V22_main_crime_date_month         0
V30_program_start_year            0
V30_program_start_month     

## trun categorical data into numeric data

In [128]:
# drop id
columus_to_drop = ['id']
df = df.drop(columus_to_drop,axis=1)

In [129]:
# detect the categorical data
num_cols = df._get_numeric_data().columns
print(num_cols)
cols = df.columns
cat_cols = list(set(cols) - set(num_cols))
print(cat_cols)

Index(['V8_age', 'V9_age_at_program_end', 'V11_criminal_record',
       'V16_violent_crime', 'V17_crime_classification',
       'V20_n_juvenile_records', 'V21_n_crime',
       'V28_days_from_crime_to_program', 'V29_program_duration',
       'V115_RECID2015_recid', 'V10_date_of_birth_year',
       'V10_date_of_birth_month', 'V22_main_crime_date_year',
       'V22_main_crime_date_month', 'V30_program_start_year',
       'V30_program_start_month', 'V31_program_end_month'],
      dtype='object')
['V6_province', 'V19_committed_crime', 'V23_territory_of_execution', 'V12_n_criminal_record', 'V15_main_crime_cat', 'V24_finished_program', 'V4_area_origin', 'V13_n_crime_cat', 'V27_program_duration_cat', 'V1_sex', 'V26_finished_measure_grouped']


In [130]:
# check the types in each categorical data
for each_col in cat_cols:
    print('---{}---'.format(each_col))
    print(df[each_col].value_counts())

---V6_province---
Barcelona    3257
Girona        594
Tarragona     449
Lleida        352
Name: V6_province, dtype: int64
---V19_committed_crime---
Lesions                                        1011
Robatori i robatori amb força en les coses      530
Furt                                            527
Danys                                           401
Contra la llibertat                             354
Robatori amb violència                          335
Robatori amb violència o intimidació            246
Trànsit                                         210
Violència familiar                              198
Robatori i furt d'ús de vehicle a motor         185
Robatori amb intimidació                        135
Atemptat contra l'autoritat                     133
Contra el patrimoni i l'ordre socioeconòmic      84
Contra l'ordre públic i interessos generals      67
Contra la llibertat sexual                       56
Contra l'honor i la intimitat                    36
Contra la salud públ

In [131]:
df = pd.get_dummies(df,columns=cat_cols)
df.head()

,V8_age,V9_age_at_program_end,V11_criminal_record,V16_violent_crime,V17_crime_classification,V20_n_juvenile_records,V21_n_crime,V28_days_from_crime_to_program,V29_program_duration,V115_RECID2015_recid,...,V27_program_duration_cat_<6 months,V27_program_duration_cat_>1 year,V1_sex_female,V1_sex_male,V26_finished_measure_grouped_ATM,V26_finished_measure_grouped_Community Service,V26_finished_measure_grouped_Internment,V26_finished_measure_grouped_MRM,V26_finished_measure_grouped_Other,V26_finished_measure_grouped_Probation
0,15,17,1,1,1,1.0,4,587.0,132,1,...,1,0,0,1,0,0,1,0,0,0
1,14,15,1,1,1,1.0,2,107.0,258,1,...,0,0,1,0,0,0,1,0,0,0
2,15,16,1,1,1,4.0,3,135.0,85,1,...,1,0,0,1,0,0,1,0,0,0
3,14,15,1,1,1,7.0,2,176.0,271,1,...,0,0,0,1,0,0,1,0,0,0
4,17,20,1,1,1,6.0,3,261.0,533,1,...,0,1,0,1,0,0,1,0,0,0


In [185]:
# somehow not allowed for xgboost
col_names = df.columns
for each in col_names:
    if '>' in each or '<' in each:
        print(each)

In [133]:
df['V27_program_duration_cat_between6monthto1year'] =df['V27_program_duration_cat_6 months < 1 year']
df['V27_program_duration_cat_lessthan6month'] =df['V27_program_duration_cat_<6 months']
df['V27_program_duration_cat_morethan1year'] =df['V27_program_duration_cat_>1 year']
df = df.drop(['V27_program_duration_cat_6 months < 1 year','V27_program_duration_cat_<6 months','V27_program_duration_cat_>1 year'],axis=1)

In [135]:
# correlated with other columns
columns_to_drop = ['V6_province_Lleida','V19_committed_crime_Violència familiar',
                  'V23_territory_of_execution_Tarragona',
                  'V12_n_criminal_record_5+',
                  'V15_main_crime_cat_Other',
                  'V24_finished_program_Tractament terapèutic ambulatori',
                  'V4_area_origin_Other',
                  'V13_n_crime_cat_3+',
                  'V1_sex_male','V26_finished_measure_grouped_Probation',
                  'V27_program_duration_cat_morethan1year']

In [136]:
df = df.drop(columns_to_drop,axis=1)

In [186]:

# save new file
df.to_csv('../catalan_data_course/catalan-juvenile-recidivism-subset-numeric.csv')  

### tree-base models

In [169]:


X = df.drop(['V115_RECID2015_recid'], axis=1)

y = df['V115_RECID2015_recid']
A_attributes = ['V1_sex_female']
y_with_A = df[['V115_RECID2015_recid','V1_sex_female']]

In [173]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_with_A, y_test_with_A = train_test_split(X, y_with_A, test_size = 0.33, random_state = 42)

In [175]:
y_train = y_train_with_A.drop(A_attributes,axis=1)
y_test = y_test_with_A.drop(A_attributes,axis=1)

In [176]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0,n_estimators=100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)



from sklearn.metrics import accuracy_score
print('Model accuracy score : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

C:\Users\12632\AppData\Local\Temp\ipykernel_2208\3568236683.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


Model accuracy score : 0.7246


In [182]:
male_X_test = X_test[X_test['V1_sex_female']==0]
male_y_test_with_A = y_test_with_A[y_test_with_A['V1_sex_female']==0]

female_X_test = X_test[X_test['V1_sex_female']==1]
female_y_test_with_A = y_test_with_A[y_test_with_A['V1_sex_female']==1]
print(len(female_X_test))
print(len(male_X_test))

Europe_X_test = X_test[X_test['V4_area_origin_Europe']==1]
Latin_America_X_test = X_test[X_test['V4_area_origin_Latin America']==1]
Maghreb_X_test = X_test[X_test['V4_area_origin_Maghreb']==1]
print(len(Europe_X_test))
print(len(Latin_America_X_test))
print(len(Maghreb_X_test))

266
1270
87
237
209


performance on men and women

In [183]:
print('Model accuracy score : {0:0.4f}'. format(accuracy_score(male_y_test_with_A['V115_RECID2015_recid'], rfc.predict(male_X_test))))

Model accuracy score : 0.7039


In [184]:
print('Model accuracy score : {0:0.4f}'. format(accuracy_score(female_y_test_with_A['V115_RECID2015_recid'], rfc.predict(female_X_test))))

Model accuracy score : 0.8233


### xgboost

In [112]:
X_train.columns

Index(['V8_age', 'V9_age_at_program_end', 'V11_criminal_record',
       'V16_violent_crime', 'V17_crime_classification',
       'V20_n_juvenile_records', 'V21_n_crime',
       'V28_days_from_crime_to_program', 'V29_program_duration',
       'V10_date_of_birth_year', 'V10_date_of_birth_month',
       'V22_main_crime_date_year', 'V22_main_crime_date_month',
       'V30_program_start_year', 'V30_program_start_month',
       'V31_program_end_month', 'V6_province_Barcelona', 'V6_province_Girona',
       'V6_province_Tarragona',
       'V19_committed_crime_Atemptat contra l'autoritat',
       'V19_committed_crime_Contra el patrimoni i l'ordre socioeconòmic',
       'V19_committed_crime_Contra l'administració de justícia',
       'V19_committed_crime_Contra l'administració pública',
       'V19_committed_crime_Contra l'honor i la intimitat',
       'V19_committed_crime_Contra l'ordre públic i interessos generals',
       'V19_committed_crime_Contra la integritat moral',
       'V19_committed_

In [113]:

from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

C:\Users\12632\.conda\envs\eegeyenet_benchmark\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\12632\.conda\envs\eegeyenet_benchmark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[14:15:13] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [114]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [115]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 70.12%


In [102]:
X_test['V1_sex_female']

4072    0
2817    0
33      0
2293    0
4125    1
       ..
1490    0
2889    0
729     0
2387    0
2178    1
Name: V1_sex_female, Length: 1536, dtype: uint8